In [1]:
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputClassifier


from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,plot_confusion_matrix,roc_auc_score
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV


import warnings
warnings.filterwarnings('ignore')

# Prediction vs Norwich (23/12/2021)

In [2]:
df=pd.read_csv('Arsenal season 2021_2022 - Sheet1 (2).csv')
drop=['pred_poss','Day','Date','Round','Prediction','GF_prediction', 'GA_prediction','Predicted_formation','pred_gk_Ratings','pred_def_ratings', 'pred_mid_ratings', 'pred_forw_ratings','Opponent','Venue','Result']


opponent=pd.get_dummies(df['Opponent'],drop_first=True)
venue=pd.get_dummies(df['Venue'],drop_first=True)
result=pd.get_dummies(df['Result'],drop_first=True)

df=pd.concat([df,opponent,venue,result],axis=1)
df=df.iloc[:18]

df=df.drop(drop,axis=1)

df['Formation'].replace('3-4-2003','3-4-3',inplace=True)
df['Formation']=df['Formation'].apply(lambda x:int(x.replace('-','')))

In [5]:
train=df.iloc[:14]
test=df.iloc[14:]

In [6]:
x_train=train.drop(['GF','GA','L','W'],axis=1)
y_train=train[['L','W']]

In [7]:
x_test=test.drop(['GF','GA','L','W'],axis=1)
y_test=test[['L','W']]

In [16]:
y_train_scoring=train[['GF','GA']]
y_test_scoring=test[['GF','GA']]

Predicting match results

In [21]:
norwich_input=[[2.033333333,0.9333333333,58,4411,7.051666667,6.963333333,7.070416667,7.343333333,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]]

In [22]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train)
lr.predict(norwich_input)

array([[0, 1]], dtype=uint8)

Predicting match scoring

In [20]:
model = LogisticRegression(C=100)
lr=MultiOutputClassifier(model)
lr.fit(x_train,y_train_scoring)
lr.predict(norwich_input)

array([[1., 0.]])

###    The model predicts Arsenal will win against Norwich with a score of 1-0.